# Model Selection

1. Data Preparation
   - Load the data
   - Split into features and target
   - Create train/test split

2. Define Evaluation Metrics
   - Accuracy, Precision, Recall, F1-score for win prediction
   - Mean Absolute Error, Mean Squared Error for score prediction

3. Model Comparison (for win prediction)
   - Logistic Regression
   - Random Forest
   - Gradient Boosting (e.g., XGBoost)
   - Support Vector Machines

4. Model Comparison (for score prediction)
   - Linear Regression
   - Decision Trees
   - Random Forest
   - Gradient Boosting

5. Cross-Validation
   - Implement k-fold cross-validation for each model

6. Hyperparameter Tuning
   - Use GridSearchCV or RandomizedSearchCV for best models

7. Final Model Selection
   - Choose the best model based on cross-validation results
   - Evaluate on the test set

8. Save Best Models
   - Save the best models and their corresponding scalers

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import sqlite3
import pandas as pd
import joblib
from datetime import datetime
from sklearn.model_selection import train_test_split

# Add the project root to the Python path
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.dirname(notebook_dir)
sys.path.append(project_root)

## 1.   Data Preparation

In [8]:
target_db_path = '../data/04_features/features_teams.db'
conn = sqlite3.connect(target_db_path)

# Read the data into a DataFrame
df_all_years = pd.read_sql_query("SELECT * FROM features_teams", conn)

# Close the connection
conn.close()

In [12]:
# Load the data
df_all_years = pd.read_parquet('../data/03_processed/preprocessed_all_years.parquet')
df_2016_plus = pd.read_parquet('../data/03_processed/preprocessed_2016_plus.parquet')

In [5]:
def split_data(df, target_column, drop_columns=None, test_size=0.2, val_size=0.2, random_state=42):
    """
    Split the data into train, validation, and test sets based on years.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame
    target_column (str): The name of the target column
    drop_columns (list): List of column names to drop from features. If None, use all columns except target and 'season'
    test_size (float): Proportion of data to use for test set
    val_size (float): Proportion of non-test data to use for validation set
    random_state (int): Random state for reproducibility
    
    Returns:
    tuple: (X_train, X_val, X_test, y_train, y_val, y_test)
    """
    
    # Sort the DataFrame by year to ensure chronological order
    df = df.sort_values('year')
    
    # Define columns to drop
    if drop_columns is None:
        drop_columns = []
    drop_columns = set(drop_columns + [target_column, 'year'])
    
    # Select feature columns (all columns except those in drop_columns)
    feature_columns = [col for col in df.columns if col not in drop_columns]
    
    # Split features (X) and target (y)
    X = df[feature_columns]
    y = df[target_column]
    
    # Get unique years
    years = df['year'].unique()
    
    # Calculate the number of years for test and validation
    n_years = len(years)
    n_test_years = max(1, int(n_years * test_size))
    n_val_years = max(1, int((n_years - n_test_years) * val_size))
    
    # Split years into train, validation, and test
    test_years = years[-n_test_years:]
    val_years = years[-(n_test_years + n_val_years):-n_test_years]
    train_years = years[:-(n_test_years + n_val_years)]
    
    # Create masks for each split
    test_mask = df['year'].isin(test_years)
    val_mask = df['year'].isin(val_years)
    train_mask = df['year'].isin(train_years)
    
    # Split the data
    X_train, y_train = X[train_mask], y[train_mask]
    X_val, y_val = X[val_mask], y[val_mask]
    X_test, y_test = X[test_mask], y[test_mask]
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [9]:
X_train, X_val, X_test, y_train, y_val, y_test = split_data(
    df_all_years,
    target_column='win',
    drop_columns=[
        'season_type',
        'team_id',
        'opponent_id',
        'team_conference',
        'opponent_conference',
        'start_date'
        ]
    )

##  3.  Model Comparison

### 3.1 Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

def create_improved_logistic_regression_model(X_train, X_val, X_test, y_train, y_val, y_test):
    # Create a pipeline with more steps and options
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree=2, include_bias=False)),
        ('logistic_regression', LogisticRegression(random_state=42, max_iter=1000))
    ])
    
    # Define parameter grid for GridSearchCV
    param_grid = {
        'poly__degree': [1],
        'logistic_regression__C': [0.01, 0.1, 1, 10],
        'logistic_regression__penalty': ['l2'],
        'logistic_regression__solver': ['lbfgs']
    }
    
    # Perform grid search
    grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    # Get the best model
    best_model = grid_search.best_estimator_
    
    # Make predictions
    train_predictions = best_model.predict(X_train)
    val_predictions = best_model.predict(X_val)
    test_predictions = best_model.predict(X_test)
    
    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, train_predictions)
    val_accuracy = accuracy_score(y_val, val_predictions)
    test_accuracy = accuracy_score(y_test, test_predictions)
    
    # Print results
    print("Best parameters:", grid_search.best_params_)
    print("\nValidation Set Classification Report:")
    print(classification_report(y_val, val_predictions))
    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    
    # Get feature importances
    feature_importances = abs(best_model.named_steps['logistic_regression'].coef_[0])
    feature_names = X_train.columns
    
    # Print top 10 feature importances
    print("\nTop 10 Feature Importances:")
    for name, importance in sorted(zip(feature_names, feature_importances), key=lambda x: x[1], reverse=True)[:10]:
        print(f"Feature '{name}': {importance:.4f}")
    
    return best_model

# Usage
improved_model = create_improved_logistic_regression_model(X_train, X_val, X_test, y_train, y_val, y_test)

Best parameters: {'logistic_regression__C': 0.01, 'logistic_regression__penalty': 'l2', 'logistic_regression__solver': 'lbfgs', 'poly__degree': 1}

Validation Set Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.68      0.68      1789
           1       0.68      0.66      0.67      1789

    accuracy                           0.67      3578
   macro avg       0.67      0.67      0.67      3578
weighted avg       0.67      0.67      0.67      3578

Training Accuracy: 0.6849
Validation Accuracy: 0.6736
Test Accuracy: 0.6482

Top 10 Feature Importances:
Feature 'is_home': 0.5039
Feature 'points_allowed_last_3': 0.2823
Feature 'points_scored_last_3': 0.2387
Feature 'conference_game': 0.2068
Feature 'rushingAttempts_last_3': 0.1565
Feature 'win_rate_last_10': 0.1515
Feature 'yardsPerRushAttempt_last_3': 0.1037
Feature 'win_rate_last_3': 0.1030
Feature 'rushingYards_last_3': 0.1027
Feature 'offense_line_yards_last_10': 0.1004


### 3.2 Random Forest

In [11]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, accuracy_score

def create_lightweight_random_forest(X_train, X_val, X_test, y_train, y_val, y_test):
    # Create a pipeline with reduced complexity
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),  # Handle null values
        ('scaler', StandardScaler()),  # Scale features
        ('rf', RandomForestClassifier(n_estimators=500, max_depth=5, random_state=42))  # Reduced parameters
    ])
    
    # Fit the model
    pipeline.fit(X_train, y_train)
    
    # Make predictions
    train_predictions = pipeline.predict(X_train)
    val_predictions = pipeline.predict(X_val)
    test_predictions = pipeline.predict(X_test)
    
    # Calculate accuracies
    train_accuracy = accuracy_score(y_train, train_predictions)
    val_accuracy = accuracy_score(y_val, val_predictions)
    test_accuracy = accuracy_score(y_test, test_predictions)
    
    # Print results
    print("\nValidation Set Classification Report:")
    print(classification_report(y_val, val_predictions))
    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    
    # Feature importance (top 10)
    feature_importance = pipeline.named_steps['rf'].feature_importances_
    feature_names = X_train.columns
    print("\nTop 10 Feature Importances:")
    for name, importance in sorted(zip(feature_names, feature_importance), key=lambda x: x[1], reverse=True)[:10]:
        print(f"Feature '{name}': {importance:.4f}")
    
    return pipeline

# Usage
lightweight_model = create_lightweight_random_forest(X_train, X_val, X_test, y_train, y_val, y_test)


Validation Set Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.58      0.63      1789
           1       0.64      0.74      0.69      1789

    accuracy                           0.66      3578
   macro avg       0.67      0.66      0.66      3578
weighted avg       0.67      0.66      0.66      3578

Training Accuracy: 0.6896
Validation Accuracy: 0.6615
Test Accuracy: 0.7195

Top 10 Feature Importances:
Feature 'is_home': 0.1085
Feature 'win_rate_last_10': 0.0790
Feature 'win_rate_last_5': 0.0669
Feature 'points_allowed_last_3': 0.0554
Feature 'win_rate_last_3': 0.0517
Feature 'points_scored_last_3': 0.0444
Feature 'win_rate_last_1': 0.0332
Feature 'offense_success_rate_last_10': 0.0245
Feature 'offense_standard_downs_success_rate_last_10': 0.0188
Feature 'points_allowed_last_1': 0.0187


### 3.3 XGBoost

In [12]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, accuracy_score

def create_xgboost_model(X_train, X_val, X_test, y_train, y_val, y_test):
    # Create a pipeline
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),  # Handle null values
        ('scaler', StandardScaler()),  # Scale features
        ('xgb', XGBClassifier(
            n_estimators=100,  # Number of boosting rounds
            max_depth=3,       # Maximum tree depth
            learning_rate=0.1, # Learning rate
            subsample=0.8,     # Subsample ratio of the training instances
            colsample_bytree=0.8, # Subsample ratio of columns when constructing each tree
            random_state=42,
            use_label_encoder=False,  # Avoid warning about label encoder
            eval_metric='logloss'     # Evaluation metric
        ))
    ])
    
    # Fit the model
    pipeline.fit(X_train, y_train)
    
    # Make predictions
    train_predictions = pipeline.predict(X_train)
    val_predictions = pipeline.predict(X_val)
    test_predictions = pipeline.predict(X_test)
    
    # Calculate accuracies
    train_accuracy = accuracy_score(y_train, train_predictions)
    val_accuracy = accuracy_score(y_val, val_predictions)
    test_accuracy = accuracy_score(y_test, test_predictions)
    
    # Print results
    print("\nValidation Set Classification Report:")
    print(classification_report(y_val, val_predictions))
    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    
    # Feature importance (top 10)
    feature_importance = pipeline.named_steps['xgb'].feature_importances_
    feature_names = X_train.columns
    print("\nTop 10 Feature Importances:")
    for name, importance in sorted(zip(feature_names, feature_importance), key=lambda x: x[1], reverse=True)[:10]:
        print(f"Feature '{name}': {importance:.4f}")
    
    return pipeline

# Usage
xgboost_model = create_xgboost_model(X_train, X_val, X_test, y_train, y_val, y_test)

/Users/colingaffney/repos/personal-projects/college_football_predictor/venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [10:40:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Validation Set Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.64      0.67      1789
           1       0.67      0.71      0.69      1789

    accuracy                           0.68      3578
   macro avg       0.68      0.68      0.68      3578
weighted avg       0.68      0.68      0.68      3578

Training Accuracy: 0.7243
Validation Accuracy: 0.6761
Test Accuracy: 0.7089

Top 10 Feature Importances:
Feature 'win_rate_last_5': 0.0767
Feature 'win_rate_last_10': 0.0613
Feature 'is_home': 0.0441
Feature 'win_rate_last_1': 0.0339
Feature 'points_allowed_last_3': 0.0275
Feature 'points_scored_last_3': 0.0207
Feature 'offense_success_rate_last_10': 0.0179
Feature 'conference_game': 0.0165
Feature 'defense_total_ppa_last_10': 0.0149
Feature 'points_scored_last_1': 0.0143
